In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as sk
from sklearn.datasets import fetch_openml

### 1. Demo: Random Forest Feature Importance for Feature Selection

In [ ]:
mnist = fetch_openml('mnist_784')

In [ ]:
print(mnist.DESCR)

In [ ]:
df = mnist.data
df.head()

In [ ]:
# combine label into dataframe
df['label'] = mnist.target
df.head()

In [ ]:
df.shape

In [ ]:
# look at features and labels separately
X = df.iloc[:,0:-1:]
y = df.iloc[:,-1]

In [ ]:
X

In [ ]:
image = X.iloc[0].values.reshape(28,28)
# plot the sample
fig = plt.figure
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
sns.heatmap(X.iloc[0].values.reshape(28,28))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X,y)

In [ ]:
rf.feature_importances_

In [ ]:
rf.feature_importances_.shape

In [ ]:
sns.heatmap(rf.feature_importances_.reshape(28,28))

In [ ]:
np.where(rf.feature_importances_ > 0.004)[0].shape

### 2. Remove constant/quasi-constant features

$$
\sigma^2 = \frac{1}{n} \sum_{i=1}^n (x^{(i)}-\bar{x})^2
$$

| F1 | F2 | F3 | F4 |
| --- | --- | --- | --- |
| 1 | 0 | 1 | 2 |
| 2 | 8 | 1 | 2 |
| 4 | 5 | 1 | 2 |
| 6 | 6 | 1 | 0 |
| 9 | 4 | 1 | 2 |


1. Features with zero or low variance do not explain the target variable in any way (i.e. no predictive power).
2. Such features can be removed by using VarianceThreshold transformer
3. It takes a threshold cut-off value. All values below that threshold value will be dropped
4. Default threshold value = 0. It drops only constant
5. A quasi-constant feature, using a threshold of 0.1 means 90% of the values are similar
6. Although not mandatory, normalizing (not standardizing) the features before applying VarianceThreshold is a good idea for exploratory purposes of fairer comparison of variance across features and set a cutoff. Otherwise variance is a running value
7. Can be applied for Categorical variables after Label/Ordinal Encoding



**Implementation**:
Used below is a public dataset from 2012 U.S. Army Anthropometric Survey: http://mreed.umtri.umich.edu/mreed/downloads/anthro/ANSUR2Distribution.zip. The zip contains CSV for male and female. Only make dataset is used for testing VarianceThreshold


In [ ]:
#Demo
from sklearn.preprocessing import OneHotEncoder

X = [['blue'], ['green'], ['blue'], ['blue'], 
     ['green'], ['red'], ['blue'], ['green']]
y = [0, 0, 1, 0, 0, 1, 0, 0]

enc = OneHotEncoder(drop='first')
enc.fit(X)
X_ohe = enc.transform(X)
X_ohe.toarray()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

sel.fit(X_ohe)
sel.transform(X_ohe).toarray()

**On real dataset**

In [ ]:
df = pd.read_csv("data/ANSUR II MALE Public.csv", encoding='latin')
df.head()

In [ ]:
df.drop(columns=["subjectid"], inplace=True)
df.head()

In [ ]:
df.shape # 107 features

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("data/ANSUR II MALE Public.csv", encoding='latin')
df.drop(columns=["subjectid"], inplace=True)

df = df.select_dtypes(include='number')
X, y = df.iloc[:, :-1], df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
X_train.shape

##### 2.1 Run with all features

Note the accuracy and time taken

In [ ]:
%%timeit
model = RandomForestRegressor(random_state=42)
_ = model.fit(X_train, y_train)

In [ ]:
model = RandomForestRegressor(random_state=42)
_ = model.fit(X_train, y_train)
print(f"Training Score: {model.score(X_train, y_train)}")
print(f"Test Score: {model.score(X_test, y_test)}")

##### 2.2 Identify quasi-constant features with Pandas

In [ ]:
x_mean = X_train.mean()
x_mean

In [ ]:
X_train_normalized = X_train / x_mean
X_test_normalized = X_test/x_mean

In [ ]:
quasi_constant_features = [feat for feat in X_train.columns if X_train_normalized[feat].var() <= 0.03]
print(quasi_constant_features)

##### 2.3. Identify quasi-constant features with VarianceThreshold

1. StandardScaler is not used because it sets variance = 1
2. Normalizer is not used because the output of the transformer will no longer be a dataframe. We will do it from scratch

In [ ]:
#from sklearn.preprocessing import Normalizer

#normalizer = Normalizer()
#X_train_normalized = normalizer.fit_transform(X_train)
#X_test_normalized = normalizer.transform(X_test)

# Normalizer is not used because the output is no longer a dataframe.
# We will do it frm scratch

In [ ]:
x_mean = X_train.mean()
x_mean

In [ ]:
X_train_normalized = X_train / x_mean
X_test_normalized = X_test/x_mean

In [ ]:
X_train_normalized.var()

**Observation**: Applying a threshold of 0.003 removes approximately 50% of the features

In [ ]:
np.where(X_train_normalized.var() > 0.003)

In [ ]:
np.where(X_train_normalized.var() > 0.003)[0].shape

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0.003)
vt.fit_transform(X_train_normalized)

mask = vt.get_support()
mask # mask tells which column to retain or remove

In [ ]:
X_train_final = X_train_normalized.loc[:, mask]
X_test_final = X_test_normalized.loc[:, mask]

In [ ]:
vt.get_feature_names_out() #show the retained features

In [ ]:
X_train_final.shape # Ensure that the number of columns is half

In [ ]:
%%timeit
model = RandomForestRegressor(random_state=42)
_ = model.fit(X_train_final, y_train)

In [ ]:
model = RandomForestRegressor(random_state=42)
_ = model.fit(X_train_final, y_train)
print(f"Training Score: {model.score(X_train_final, y_train)}")
print(f"Test Score: {model.score(X_test_final, y_test)}")

**Observations**: 
1. Time taken for model training is reduced by more than half.
2. Model accuracy is not at all affected on both train and test 

##### 2.4 Identify quasi-constant features with Feature Engine

1. !pip install feature-engine
2. Feature-engine is an open source Python library with the most exhaustive battery of transformers to engineer features for use in machine learning models. Feature-engine simplifies and streamlines the implementation of and end-to-end feature engineering pipeline, by allowing the selection of feature subsets within its transformers, and returning dataframes for easy data exploration. Feature-engine’s transformers preserve Scikit-learn functionality with the methods fit() and transform() to learn parameters from and then transform data
3. https://feature-engine.trainindata.com/en/latest/
4. https://feature-engine.trainindata.com/en/latest/api_doc/index.html
5. https://feature-engine.trainindata.com/en/latest/user_guide/selection/DropConstantFeatures.html
6. DropConstantFeatures takes a tolerance level. E.g. tol=.7 to remove features that show the same value in more than 70% of the observations
7. Here is a good executive summary of available product features: https://trainindata.medium.com/feature-engine-a-new-open-source-python-package-for-feature-engineering-29a0ab88ea7c

In [ ]:
from sklearn.model_selection import train_test_split
from feature_engine.datasets import load_titanic
from feature_engine.selection import DropConstantFeatures

X, y = load_titanic(return_X_y_frame=True, handle_missing=True,)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train.shape # 13 features

In [ ]:
X_train['embarked'].value_counts(normalize = True)

**Observation** More than 70% of values in the embarked feature are same viz S

In [ ]:
transformer = DropConstantFeatures(tol=0.7)
transformer.fit(X_train)
transformer.features_to_drop_

In [ ]:
X_train_vars = transformer.transform(X_train)
X_test_vars = transformer.transform(X_test)

In [ ]:
X_train_vars.shape # 4 features dropped

In [ ]:
transformer.get_support() #this is similar to sklearn VarianceThreshold 

### 3. Highly correlated features

While high correlation between feature and target is good, high correlation between features is not good

##### 3.1 Visualize heatmap

**Warning**
1. Correlation is a linear measure. 
2. There can be non linear relation. That will not be measured by pearson's correlation 
3. Use mutual information to measure that

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing["target"] = housing.target
df_housing

In [ ]:
# Increase the size of the heatmap.
plt.figure(figsize=(16, 6))
# Store heatmap object in a variable to easily access it when 
# you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and 
# set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df_housing.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

**Observations**: AveRooms and AveBedrooms are highly correlated

In [ ]:
plt.scatter(df_housing["AveRooms"], df_housing["AveBedrms"], alpha=0.3, color="purple")
plt.xlabel("var_8")
plt.ylabel("var_6")
plt.show()

A diverging color palette that has markedly different colors at the two ends of the value-range with a pale, almost colorless midpoint, works much better with correlation heatmaps than the default colormap

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df_housing.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

#plt.savefig('heatmap.png', dpi=300, bbox_inches='tight')

Removing target column with triu-1 lets us focus only on features

In [ ]:
plt.figure(figsize=(16, 6))

# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_housing.corr(), dtype=np.bool))
heatmap = sns.heatmap(df_housing.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

##### 3.2 Spearman correlation

In this case Spearman correlation does not deviate much from pearson for Latitude and Longitude. However Spearmzn correlation between AveRooms and AvgBedrooms very less. Correpsonding pearson correlation was high. This indicates either one of those two columns contain outliers or some other reason

In [ ]:
plt.figure(figsize=(16, 6))

# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_housing.corr(), dtype=np.bool))
heatmap = sns.heatmap(df_housing.corr(method="spearman"), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

#### 3.3 Using Feature Engine to drop correlated features

In [ ]:
print(f"Number of features: {df_housing.iloc[:,:-1].shape}")

In [ ]:
from feature_engine.selection import DropCorrelatedFeatures

sel = DropCorrelatedFeatures(method="pearson", threshold=0.8)
sel.fit(df_housing.iloc[:,:-1])

In [ ]:
sel.correlated_feature_sets_

In [ ]:
sel.features_to_drop_

In [ ]:
X_no_corr = sel.transform(df_housing.iloc[:,:-1]) # 2 columns are dropped 
X_no_corr.shape

##### 3.4 Remove correlated features and retain the best correlated feature with target

1. Removing correlated features in previous section removed all but one correlated feature.
2. But there is no guarantee that the retained feature is the one that has best correlation with target variable
3. In a given dataset, we can find groups of features that are correlated among themselves or to a given feature. From every one of these groups, we can retain the feature that brings most value to the
predictive model and remove the rest
4. FeatureEngine provides a class called SmartCorrelatedSelection for this purpose. 
5. SmartCorrelatedSelection identifies features with a correlation coefficient higher configured value, then retain the feature with the highest importance from each group of correlated variables.
6. In this sense it is a wrapper method, but the goal it achieves is similar to Filter method

In [ ]:
X = df_housing.iloc[:,:-1]
y = df_housing.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from feature_engine.selection import SmartCorrelatedSelection

sel = SmartCorrelatedSelection(method="pearson", threshold=0.8, 
                               selection_method="model_performance",
                               estimator=RandomForestRegressor(n_estimators=5, random_state=10),
                               scoring="r2", cv=3)

In [ ]:
sel.fit(X_train, y_train)

In [ ]:
sel.features_to_drop_

In [ ]:
X_train_uncorrel = sel.transform(X_train)
X_test_uncorrel = sel.transform(X_test)

In [ ]:
X_train_uncorrel.shape

### 4. Filter methods

1. All filter methods are supervised.
2. Implemented froms scratch and with SelectKBest

##### 4.1 Data preprocessing of Titanic

In [ ]:
# titanic=sns.load_dataset('titanic')

titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

In [ ]:
df_titanic = titanic[0]
df_titanic['survived'] = titanic[1] 
df_titanic.head()

In [ ]:
df_titanic.shape

In [ ]:
df_titanic.info()

In [ ]:
print(f"Unique values for feature cabin {df_titanic['cabin'].nunique()}")
print(f"Unique values for feature body {df_titanic['body'].nunique()}")
print(f"Unique values for feature boat {df_titanic['boat'].nunique()}")


In [ ]:
df_titanic['survived'].isna().sum()

In [ ]:
df_titanic.drop(columns=["name", "ticket", "cabin", "body", "home.dest", "boat"], inplace=True)
df_titanic.head()

In [ ]:
df_titanic.info()

In [ ]:
df_series = df_titanic["embarked"].isna()
np.where(df_series)

In [ ]:
df_titanic.drop(np.where(df_titanic["embarked"].isna())[0], inplace=True)
df_titanic.drop(np.where(df_titanic["fare"].isna())[0], inplace=True)
df_titanic.shape

In [ ]:
df_titanic['embarked'].value_counts()

In [ ]:
X = df_titanic.iloc[:,:-1:]
y = df_titanic.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
sex_train_encoded = lbl_encoder.fit_transform(X_train["sex"])
sex_test_encoded = lbl_encoder.transform(X_test["sex"])

lbl_encoder2 = LabelEncoder()
embark_train_encoded = lbl_encoder.fit_transform(X_train["embarked"])
embark_test_encoded = lbl_encoder.transform(X_test["embarked"])

tgt_encoder = LabelEncoder()
y_train_encoded = tgt_encoder.fit_transform(y_train)
y_test_encoded = tgt_encoder.transform(y_test)

In [ ]:
X_train_new = np.hstack( 
    (X_train.iloc[:,0:1].to_numpy(), sex_train_encoded.reshape(-1,1), 
     X_train.iloc[:,2:6].to_numpy(), embark_train_encoded.reshape(-1,1)))
X_test_new = np.hstack(
    (X_test.iloc[:,0:1].to_numpy(), sex_test_encoded.reshape(-1,1), 
     X_test.iloc[:,2:6].to_numpy(), embark_test_encoded.reshape(-1,1)))

In [ ]:
X_train_new

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=7, weights='uniform', metric='nan_euclidean')
X_train_imputed = imputer.fit_transform(X_train_new)
X_train_imputed[0:5]

##### 4.2 Finding feature importance with mutual information

In [ ]:
from scipy.stats.contingency import crosstab
from sklearn.metrics import mutual_info_score

In [ ]:
# method 1 direct
mi = mutual_info_score(y_train, X_train['sex'])
mi

In [ ]:
# method 2 get crosstab first and then mi
c = crosstab(y_train, X_train['sex'])
c

In [ ]:
mutual_info_score(labels_true=None, labels_pred=None, contingency = c[1])

**Conclusion** There is mutual information between sex and survived

##### 4.3 Find mutual information in the most direct manner between all features and categorical target


In [ ]:
from sklearn.feature_selection import mutual_info_classif

mi_score = mutual_info_classif(X_train_imputed, y_train, n_neighbors=10, random_state=22)
sorted_idx = np.argsort(mi_score)
mi_scoredf = pd.DataFrame(
    mi_score[sorted_idx[::-1]], 
    index=X_train.columns[sorted_idx[::-1]], 
    columns=['mi_score'])
plt.barh(
    X_train.columns[sorted_idx], 
    mi_score[sorted_idx])
plt.xlabel("Mutual Information Score")

##### 4.4 Mutual Information between numerical/categorical feature and a numerical feature

In [ ]:
from sklearn.feature_selection import mutual_info_regression

#fare and age
mutual_info_regression(X_train_imputed[:,5].reshape(-1,1), 
                       X_train_imputed[:,2], discrete_features=[False])

##### 4.5 Find feature importance with chi square test

In [ ]:
c = pd.crosstab(y_train, X_train['sex'])
c

In [ ]:
from scipy.stats import chi2_contingency

chi2_contingency(c)

In [ ]:
chi_ls = []
for feature in X_train.columns: # create contingency table
    c = pd.crosstab(y_train, X_train[feature])
    # chi-square test
    p_value = chi2_contingency(c)[1]
    chi_ls.append(p_value)

pd.Series(chi_ls, index=X_train.columns).sort_values(ascending=True).plot.bar(rot=45)
plt.ylabel("p value")
plt.title("Feature importance based on chi-square test")

In [ ]:
selected = pd.Series(chi_ls, index=X_train.columns).sort_values(ascending=True)[0:3].index
selected

##### 4.6 Feature Selection with SelectKBest and scoring = chisquared

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [ ]:
kbest = SelectKBest(score_func=chi2, k=4)
fit = kbest.fit(X_train_imputed, y_train)
fit.scores_

In [ ]:
kbest.get_feature_names_out()

In [ ]:
df_titanic.columns[[0,1,5,6]]

##### 4.6 Feature Selection with SelectKBest and scoring = mutual_info_classification

In [ ]:
kbest = SelectKBest(score_func=mutual_info_classif, k=4)
fit = kbest.fit(X_train_imputed, y_train)
fit.scores_

In [ ]:
kbest.get_feature_names_out() #notice the difference between this and chisquared based feature selection

##### 4.4 Apply chisquared to wine dataset


In [ ]:
from sklearn.datasets import load_wine

X,y=load_wine(return_X_y=True)

# k = 4 tells four top features to be selected
# Score function Chi2 tells the feature to be selected using Chi Square
wine_kbest = SelectKBest(score_func=chi2, k=4)
_ = wine_kbest.fit(X, y)

wine_kbest.scores_

In [ ]:
wine_kbest.get_feature_names_out()

### 5. Feature Selection with Lasso Regression

1. For Lasso with Linear Regression, see linear_regression_reference.ipynb
2. Below is code for logistic regression with L1

In [ ]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

# if the Wine dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:

# df_wine = pd.read_csv('wine.data', header=None)


df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

In [ ]:
df_wine.shape

In [ ]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', multi_class='ovr')
# Note that C=1.0 is the default. You can increase
# or decrease it to make the regulariztion effect
# stronger or weaker, respectively.
lr.fit(X_train_std, y_train)
print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_test_std, y_test))

In [ ]:
lr.intercept_

In [ ]:
lr.coef_

In [ ]:
lr.coef_[lr.coef_!=0].shape

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
    
colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange']

weights, params = [], []
for c in np.arange(-4., 6.):
    lr = LogisticRegression(penalty='l1', C=10.**c, solver='liblinear', 
                            multi_class='ovr', random_state=0)
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    plt.plot(params, weights[:, column],
             label=df_wine.columns[column + 1],
             color=color)
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10**(-5), 10**5])
plt.ylabel('Weight coefficient')
plt.xlabel('C (inverse regularization strength)')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', 
          bbox_to_anchor=(1.38, 1.03),
          ncol=1, fancybox=True)
plt.savefig('lasso-path.pdf', dpi=300, 
            bbox_inches='tight', pad_inches=0.2)
plt.show()

### 6. Feature Selection with Feature Importance in Decision Tree

1. DIY
2. Using sklearn
 

In [ ]:
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=5, n_classes=2,
                               n_features=2, n_informative=2, n_redundant=0,
                               random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X,y)

In [ ]:
from sklearn.tree import plot_tree
plot_tree(clf)

**Exercise: Calculate feature importance using the formula**

**Calculate feature importance using sklearn**

In [ ]:
clf.feature_importances_

### 7. How Feature Importance is calculated in Random Forest

Run the relevant section from bagging_ensemble_randomforest.ipynb

### 8. Problems with Tree based default feature importance

1. Inflated feature importance for numerical feature
2. Inflated feature importance for categorical feature with high cardinality

In [ ]:
titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
df_titanic = titanic[0]
df_titanic['survived'] = titanic[1] 
df_titanic.drop(columns=["name", "ticket", "cabin", "body", "home.dest", "boat"], inplace=True)
df_titanic.drop(np.where(df_titanic["embarked"].isna())[0], inplace=True)
df_titanic.drop(np.where(df_titanic["fare"].isna())[0], inplace=True)

In [ ]:
X = df_titanic.iloc[:,:-1:]
y = df_titanic.iloc[:,-1]

In [ ]:
rng = np.random.RandomState(seed=42)
X["random_cat"] = rng.randint(3, size=X.shape[0])
X["random_num"] = rng.randn(X.shape[0])

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

lbl_encoder = LabelEncoder()
sex_train_encoded = lbl_encoder.fit_transform(X_train["sex"])
sex_test_encoded = lbl_encoder.transform(X_test["sex"])

lbl_encoder2 = LabelEncoder()
embark_train_encoded = lbl_encoder.fit_transform(X_train["embarked"])
embark_test_encoded = lbl_encoder.transform(X_test["embarked"])

tgt_encoder = LabelEncoder()
y_train_encoded = tgt_encoder.fit_transform(y_train)
y_test_encoded = tgt_encoder.transform(y_test)

In [ ]:
X_train_new = np.hstack( 
    (X_train.iloc[:,0:1].to_numpy(), sex_train_encoded.reshape(-1,1), 
     X_train.iloc[:,2:6].to_numpy(), embark_train_encoded.reshape(-1,1)))
X_test_new = np.hstack(
    (X_test.iloc[:,0:1].to_numpy(), sex_test_encoded.reshape(-1,1), 
     X_test.iloc[:,2:6].to_numpy(), embark_test_encoded.reshape(-1,1)))
	 
	 
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=7, weights='uniform', metric='nan_euclidean')
X_train_imputed = imputer.fit_transform(X_train_new)
X_test_imputed = imputer.fit_transform(X_test_new)
X_train_imputed[0:5]

In [ ]:
X_train_imputed = np.hstack( (X_train_imputed, X_train.loc[:, ["random_cat", "random_num"]].to_numpy()))
X_test_imputed = np.hstack( (X_test_imputed, X_test.loc[:, ["random_cat", "random_num"]].to_numpy()))

X_train_imputed[0:5]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 1,
         oob_score=True,
         random_state = 42)
rf.fit(X_train_imputed, y_train)

print(f"RF train accuracy: {rf.score(X_train_imputed, y_train):.3f}")
print(f"RF test accuracy: {rf.score(X_test_imputed, y_test):.3f}")

In [ ]:
from matplotlib.pyplot import figure
feat_importances = pd.Series(rf.feature_importances_, index = X_train.columns).sort_values(ascending = True)
feat_importances.plot(kind = 'barh')

### 9. Permutation based feature importance

In [ ]:
from sklearn.inspection import permutation_importance

#calculate permutation importance for test data 
result_test = permutation_importance(
    rf, X_test_imputed, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_test = result_test.importances_mean.argsort()
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx_test].T,
    columns=X.columns[sorted_importances_idx_test],
)

In [ ]:
#calculate permutation importance for training data 
result_train = permutation_importance(
    rf, X_train_imputed, y_train, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_train = result_train.importances_mean.argsort()
importances_train = pd.DataFrame(
    result_train.importances[sorted_importances_idx_train].T,
    columns=X.columns[sorted_importances_idx_train],
)

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))

importances_test.plot.box(vert=False, whis=10, ax = axs[0])
axs[0].set_title("Permutation Importances (test set)")
axs[0].axvline(x=0, color="k", linestyle="--")
axs[0].set_xlabel("Decrease in accuracy score")
axs[0].figure.tight_layout()

importances_train.plot.box(vert=False, whis=10, ax = axs[1])
axs[1].set_title("Permutation Importances (train set)")
axs[1].axvline(x=0, color="k", linestyle="--")
axs[1].set_xlabel("Decrease in accuracy score")
axs[1].figure.tight_layout()

##### 9.2 Drop Column variant

In [ ]:
from sklearn.base import clone
def dropcol_importances(rf, X_train, y_train):
    rf_ = clone(rf)
    rf_.random_state = 42
    rf_.fit(X_train, y_train)
    
    #use out of bag error as performance measurement
    baseline = rf_.oob_score_
    imp = []
    for col in X_train.columns:
        X = X_train.drop(col, axis=1)
        rf_ = clone(rf)
        rf_.random_state = 42
        rf_.fit(X, y_train)
        o = rf_.oob_score_
        imp.append(baseline - o)
    imp = np.array(imp)
    I = pd.DataFrame(
            data={'Feature':X_train.columns,
                  'Importance':imp})
    I = I.set_index('Feature')
    I = I.sort_values('Importance', ascending=True)
    return I

In [ ]:
df_titanic_train = pd.DataFrame(data=X_train_imputed, columns=X_train.columns)
imp = dropcol_importances(rf, df_titanic_train, y_train)
imp.plot(kind = 'barh')

### 10. Recursive Feature Elimination (RFE)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE


lr = LogisticRegression(solver='liblinear', random_state=123)

rfe = RFE(estimator=lr, n_features_to_select=5, step=1)
rfe.fit(X_train_imputed, y_train)

X_train_sub = rfe.transform(X_train_imputed)

**Which features got selected?**

In [ ]:
rfe.support_

In [ ]:
X_train.columns[rfe.support_]

##### 10. RFE as part of pipeline

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(RFE(estimator=lr, step=1),
                     KNeighborsClassifier())


parameters = {'rfe__n_features_to_select': range(1, 13), 
              'kneighborsclassifier__n_neighbors': range(1, 10) }

grid = GridSearchCV(pipe, param_grid=parameters, cv=10, n_jobs=-1)
grid.fit(X_train_imputed, y_train)

print('Best params:', grid.best_params_)
print('Best accuracy:', grid.best_score_)

In [ ]:
grid.best_estimator_.score(X_test_imputed, y_test)

In [ ]:
# alternate implementation to compare performance
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_imputed, y_train)
knn.score(X_test_imputed, y_test)

**RFE add-on with Yellow bricks**

Not working as of now due to dataset issue

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedKFold

# from yellowbrick.model_selection import rfecv
# from yellowbrick.datasets import load_bikeshare

In [ ]:
# # Load classification dataset
# X, y = load_bikeshare()

# cv = StratifiedKFold(5)
# visualizer = rfecv(RandomForestClassifier(), X=X, y=y, cv=cv, scoring='f1_weighted')